In [1]:
import pandas as pd

In [2]:
train=pd.read_csv("train.csv")


In [35]:
train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
y=train.label
train.drop('label', axis = 1, inplace = True)


In [33]:
test=pd.read_csv("test.csv")
test.drop('id', axis = 1, inplace = True)


In [34]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# By XGboost


In [11]:
import xgboost as xgb
xgb1 = xgb.sklearn.XGBClassifier(    
    learning_rate =0.1,
    n_estimators=50,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softmax',
    nthread=4,
    scale_pos_weight=1,
    seed=27)

In [57]:
from sklearn.model_selection import GridSearchCV
model=modelfit(xgb1, train, y)

In [56]:
def modelfit(alg, train_data_features, train_labels,useTrainCV=True, cv_folds=2):

    if useTrainCV:
        params=alg.get_xgb_params()
        xgb_param=dict([(key,[params[key]]) for key in params])

        boost = xgb.sklearn.XGBClassifier()
        cvresult = GridSearchCV(boost,xgb_param,cv=cv_folds)
        cvresult.fit(train,y)
        alg=cvresult.best_estimator_


    #Fit the algorithm on the data
    model=alg.fit(train_data_features, train_labels)
    return model

In [58]:
#Predict training set:
pred=model.predict(test)
data2 = pd.read_csv('sample_submission.csv')
data2['classify'] = pred
print("\nModel Report")
data2.to_csv('Output_XGboost.csv', index = False)


Model Report


# By USing the PCA......

In [59]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.2, random_state=0)


from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(train, y)

C:\ProgramData\Anaconda4\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [60]:
# Predicting the Test set results
pred1 = classifier.predict(test)

In [61]:
pred1

array([3, 0, 2, ..., 1, 1, 3], dtype=int64)

In [62]:
data3 = pd.read_csv('sample_submission.csv')
data3['classify'] = pred1
data3.to_csv('Output_PCA.csv', index = False)

In [ ]:
Dataset link:
https://www.kaggle.com/c/Kannada-MNIST